In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from skimage import feature

In [2]:
# Function to load images and labels from a dataset folder
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for label in label_names:
        label_path = os.path.join(dataset_path, label)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            images.append(image)
            labels.append(label)
    return images, labels

# Load dataset
dataset_path = 'Fish Disease Dataset'  # Replace with your actual dataset path
images, labels = load_dataset(dataset_path)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def augment_images(images, labels, augment_size=1000):
    augmented_images = []
    augmented_labels = []
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    for image, label in zip(images, labels):
        image = np.expand_dims(image, axis=0)
        label = np.expand_dims(label, axis=0)
        aug_iter = datagen.flow(image, batch_size=1)
        for i in range(augment_size):
            augmented_image = aug_iter.next()[0].astype(np.uint8)
            augmented_images.append(augmented_image)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

In [4]:
# Convert images to grayscale
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]

In [5]:
# Convert images to grayscale
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]

In [6]:
# Apply LBP
def apply_lbp(image, num_points=24, radius=8):
    lbp = local_binary_pattern(image, num_points, radius, method='uniform')
    return lbp

# Apply LDP
def apply_ldp(image):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    magnitude, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    angle_bins = np.int32(8 * angle / 360)  # Quantize the angle to 8 bins
    return angle_bins

In [7]:
# Compute histogram
def compute_histogram(image, bins=16):
    hist, _ = np.histogram(image.ravel(), bins=bins, range=(0, bins))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

In [9]:
# Apply BWT (Block-wise Truncation)
def apply_bwt(image, blocks=(8, 8)):
    features = []
    h, w = image.shape
    (bx, by) = blocks
    xsteps = np.linspace(0, w, bx + 1, dtype=int)
    ysteps = np.linspace(0, h, by + 1, dtype=int)
    for i in range(1, len(ysteps)):
        for j in range(1, len(xsteps)):
            block = image[ysteps[i-1]:ysteps[i], xsteps[j-1]:xsteps[j]]
            hist = np.histogram(block, bins=256, range=(0, 256))[0] / (block.shape[0] * block.shape[1])
            features.extend(hist)
    return features

# Apply Hough Transform
def apply_hough(image):
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    return len(lines) if lines is not None else 0

In [10]:
# Extract features from the dataset
lbp_features_list = []
ldp_features_list = []
bwt_features_list = []
hough_features_list = []

for gray_image in gray_images:
    # Apply LBP
    lbp_image = apply_lbp(gray_image)
    lbp_hist_features = compute_histogram(lbp_image, bins=16)
    lbp_features_list.append(lbp_hist_features)
    
    # Apply LDP
    ldp_image = apply_ldp(gray_image)
    ldp_hist_features = compute_histogram(ldp_image, bins=16)
    ldp_features_list.append(ldp_hist_features)
    
    # Apply BWT
    bwt_features = apply_bwt(gray_image)
    bwt_features_list.append(bwt_features)
    
    # Apply Hough Transform
    hough_features = apply_hough(gray_image)
    hough_features_list.append(hough_features)

In [11]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [12]:
# Function to train and evaluate classifiers
def train_and_evaluate_model(X_train, X_test, y_train, y_test, classifier):
    clf = classifier
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Classifier: {clf.__class__.__name__}")
    print(f"Classification Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

In [13]:
# Split the data into training and testing sets for LBP features
X_train, X_test, y_train, y_test = train_test_split(lbp_features_list, numeric_labels, test_size=0.2, random_state=42)
print("Evaluating LBP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)

Evaluating LBP Features
Classifier: SVC
Classification Accuracy: 0.6229508196721312
Precision: 0.624799704178479
Recall: 0.6229508196721312
F1 Score: 0.6094733170561564
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.79      0.69        33
           1       0.63      0.43      0.51        28

    accuracy                           0.62        61
   macro avg       0.63      0.61      0.60        61
weighted avg       0.62      0.62      0.61        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.8524590163934426
Precision: 0.8532362916902204
Recall: 0.8524590163934426
F1 Score: 0.8526183049047588
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86        33
           1       0.83      0.86      0.84        28

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [14]:
# Split the data into training and testing sets for LDP features
X_train, X_test, y_train, y_test = train_test_split(ldp_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating LDP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)


Evaluating LDP Features
Classifier: SVC
Classification Accuracy: 0.6557377049180327
Precision: 0.6707440100882723
Recall: 0.6557377049180327
F1 Score: 0.6544410484393812
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.58      0.64        33
           1       0.60      0.75      0.67        28

    accuracy                           0.66        61
   macro avg       0.67      0.66      0.66        61
weighted avg       0.67      0.66      0.65        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.8032786885245902
Precision: 0.8106274731486716
Recall: 0.8032786885245902
F1 Score: 0.8034902168164992
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.76      0.81        33
           1       0.75      0.86      0.80        28

    accuracy                           0.80        61
   macro avg       0.81      0.81      0.80        61
weighted avg       

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [15]:
# Combine LBP and LDP features
combined_features_list = [np.hstack((lbp, ldp)) for lbp, ldp in zip(lbp_features_list, ldp_features_list)]

# Split the data into training and testing sets for combined features
X_train, X_test, y_train, y_test = train_test_split(combined_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating Combined LBP and LDP Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating Combined LBP and LDP Features
Classifier: SVC
Classification Accuracy: 0.7540983606557377
Precision: 0.7585404547858277
Recall: 0.7540983606557377
F1 Score: 0.7544952963124677
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        33
           1       0.71      0.79      0.75        28

    accuracy                           0.75        61
   macro avg       0.75      0.76      0.75        61
weighted avg       0.76      0.75      0.75        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.9344262295081968
Precision: 0.9344262295081968
Recall: 0.9344262295081968
F1 Score: 0.9344262295081968
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        33
           1       0.93      0.93      0.93        28

    accuracy                           0.93        61
   macro avg       0.93      0.93      0.93        61
we

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: LogisticRegression
Classification Accuracy: 0.7540983606557377
Precision: 0.7585404547858277
Recall: 0.7540983606557377
F1 Score: 0.7544952963124677
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        33
           1       0.71      0.79      0.75        28

    accuracy                           0.75        61
   macro avg       0.75      0.76      0.75        61
weighted avg       0.76      0.75      0.75        61

Classifier: GaussianNB
Classification Accuracy: 0.819672131147541
Precision: 0.8242199894235854
Recall: 0.819672131147541
F1 Score: 0.8199632172958097
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.79      0.83        33
           1       0.77      0.86      0.81        28

    accuracy                           0.82        61
   macro avg       0.82      0.82      0.82        61
weighted avg       0.82      0.82      0.82

In [16]:
# Split the data into training and testing sets for BWT features
X_train, X_test, y_train, y_test = train_test_split(bwt_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating BWT Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)


Evaluating BWT Features
Classifier: SVC
Classification Accuracy: 0.9180327868852459
Precision: 0.9215300546448087
Recall: 0.9180327868852459
F1 Score: 0.9174948560362566
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93        33
           1       0.96      0.86      0.91        28

    accuracy                           0.92        61
   macro avg       0.92      0.91      0.92        61
weighted avg       0.92      0.92      0.92        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.9016393442622951
Precision: 0.9016393442622951
Recall: 0.9016393442622951
F1 Score: 0.9016393442622951
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        33
           1       0.89      0.89      0.89        28

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.8688524590163934
Precision: 0.8697531976220501
Recall: 0.8688524590163934
F1 Score: 0.8683524411586129
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        33
           1       0.88      0.82      0.85        28

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.7704918032786885
Precision: 0.7728891239203243
Recall: 0.7704918032786885
F1 Score: 0.7708627755794234
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.76      0.78        33
           1       0.73      0.79      0.76        28

    accuracy                           0.77        61
   macro avg       0.77      0.77      0.77        61
weighted avg       0.77      

In [17]:
print("Shape of X_train:", np.array(X_train).shape)
print("Shape of y_train:", np.array(y_train).shape)


Shape of X_train: (244, 16384)
Shape of y_train: (244,)


In [18]:
hough_features_list = np.array(hough_features_list).reshape(-1, 1)


In [19]:
# Split the data into training and testing sets for combined features
combined_features_list = [np.hstack((lbp, ldp, bwt, hough)) for lbp, ldp, bwt, hough in
                          zip(lbp_features_list, ldp_features_list, bwt_features_list, hough_features_list)]
X_train, X_test, y_train, y_test = train_test_split(combined_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating Combined LBP, LDP, BWT, and Hough Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating Combined LBP, LDP, BWT, and Hough Features
Classifier: SVC
Classification Accuracy: 0.9344262295081968
Precision: 0.9415152857775809
Recall: 0.9344262295081968
F1 Score: 0.9337776977121239
Classification Report:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        33
           1       1.00      0.86      0.92        28

    accuracy                           0.93        61
   macro avg       0.95      0.93      0.93        61
weighted avg       0.94      0.93      0.93        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.8852459016393442
Precision: 0.8946243013092245
Recall: 0.8852459016393442
F1 Score: 0.8836613380055325
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.97      0.90        33
           1       0.96      0.79      0.86        28

    accuracy                           0.89        61
   macro avg       0.90      0.88      0.88

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9180327868852459
Precision: 0.9187040700960996
Recall: 0.9180327868852459
F1 Score: 0.9181212805026437
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.92        33
           1       0.90      0.93      0.91        28

    accuracy                           0.92        61
   macro avg       0.92      0.92      0.92        61
weighted avg       0.92      0.92      0.92        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.9016393442622951
Precision: 0.909624081401922
Recall: 0.9016393442622951
F1 Score: 0.9017451084082495
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.85      0.90        33
           1       0.84      0.96      0.90        28

    accuracy                           0.90        61
   macro avg       0.90      0.91      0.90        61
weighted avg       0.91      0

In [20]:
# Function to extract features from RGB channels
def extract_color_features(image):
    channels = cv2.split(image)
    features = []
    for channel in channels:
        lbp_image = apply_lbp(channel)
        lbp_hist_features = compute_histogram(lbp_image, bins=16)
        ldp_image = apply_ldp(channel)
        ldp_hist_features = compute_histogram(ldp_image, bins=16)
        features.extend(np.hstack((lbp_hist_features, ldp_hist_features)))
    return features

# Extract features from RGB channels
rgb_features_list = [extract_color_features(image) for image in images]

In [21]:
# Split the data into training and testing sets for RGB features
X_train, X_test, y_train, y_test = train_test_split(rgb_features_list, numeric_labels, test_size=0.2, random_state=42)
print("\nEvaluating RGB Channel Features")
for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
            GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
            DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
    train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating RGB Channel Features
Classifier: SVC
Classification Accuracy: 0.7540983606557377
Precision: 0.7585404547858277
Recall: 0.7540983606557377
F1 Score: 0.7544952963124677
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        33
           1       0.71      0.79      0.75        28

    accuracy                           0.75        61
   macro avg       0.75      0.76      0.75        61
weighted avg       0.76      0.75      0.75        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.9344262295081968
Precision: 0.9344262295081968
Recall: 0.9344262295081968
F1 Score: 0.9344262295081968
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        33
           1       0.93      0.93      0.93        28

    accuracy                           0.93        61
   macro avg       0.93      0.93      0.93        61
weighted av

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9672131147540983
Precision: 0.9693989071038251
Recall: 0.9672131147540983
F1 Score: 0.9672661107970605
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        33
           1       0.93      1.00      0.97        28

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.97        61
weighted avg       0.97      0.97      0.97        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.8524590163934426
Precision: 0.8524411586128076
Recall: 0.8524590163934426
F1 Score: 0.8522187868407589
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.87        33
           1       0.85      0.82      0.84        28

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      

In [22]:
# Function to convert to different color spaces and extract features
def extract_color_space_features(image, color_space):
    if color_space == 'HSV':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif color_space == 'LAB':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif color_space == 'YCbCr':
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    else:
        raise ValueError("Unsupported color space")
    return extract_color_features(converted_image)

# Extract features from different color spaces
color_spaces = ['HSV', 'LAB', 'YCbCr']
for color_space in color_spaces:
    color_space_features_list = [extract_color_space_features(image, color_space) for image in images]
    X_train, X_test, y_train, y_test = train_test_split(color_space_features_list, numeric_labels, test_size=0.2, random_state=42)
    print(f"\nEvaluating {color_space} Color Space Features")
    for clf in [SVC(kernel='linear'), RandomForestClassifier(n_estimators=100, random_state=42),
                GradientBoostingClassifier(n_estimators=100, random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
                DecisionTreeClassifier(random_state=42), LogisticRegression(), GaussianNB()]:
        train_and_evaluate_model(X_train, X_test, y_train, y_test, clf)



Evaluating HSV Color Space Features
Classifier: SVC
Classification Accuracy: 0.7868852459016393
Precision: 0.8064312736443884
Recall: 0.7868852459016393
F1 Score: 0.7860825537958075
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        33
           1       0.71      0.89      0.79        28

    accuracy                           0.79        61
   macro avg       0.80      0.79      0.79        61
weighted avg       0.81      0.79      0.79        61

Classifier: RandomForestClassifier
Classification Accuracy: 0.9672131147540983
Precision: 0.9690866510538642
Recall: 0.9672131147540983
F1 Score: 0.9670881102896532
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       1.00      0.93      0.96        28

    accuracy                           0.97        61
   macro avg       0.97      0.96      0.97        61
weighte

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9508196721311475
Precision: 0.9514379592990391
Recall: 0.9508196721311475
F1 Score: 0.9508727683015862
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95        33
           1       0.93      0.96      0.95        28

    accuracy                           0.95        61
   macro avg       0.95      0.95      0.95        61
weighted avg       0.95      0.95      0.95        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.9016393442622951
Precision: 0.9028643487659881
Recall: 0.9016393442622951
F1 Score: 0.9012643308689595
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        33
           1       0.92      0.86      0.89        28

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9508196721311475
Precision: 0.9549180327868853
Recall: 0.9508196721311475
F1 Score: 0.9504969136217538
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        33
           1       1.00      0.89      0.94        28

    accuracy                           0.95        61
   macro avg       0.96      0.95      0.95        61
weighted avg       0.95      0.95      0.95        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.9508196721311475
Precision: 0.955579058699101
Recall: 0.9508196721311475
F1 Score: 0.9508990592624936
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        33
           1       0.90      1.00      0.95        28

    accuracy                           0.95        61
   macro avg       0.95      0.95      0.95        61
weighted avg       0.96      0

C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Classifier: AdaBoostClassifier
Classification Accuracy: 0.9836065573770492
Precision: 0.9841718485019786
Recall: 0.9836065573770492
F1 Score: 0.9836242561005287
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        33
           1       0.97      1.00      0.98        28

    accuracy                           0.98        61
   macro avg       0.98      0.98      0.98        61
weighted avg       0.98      0.98      0.98        61

Classifier: DecisionTreeClassifier
Classification Accuracy: 0.9508196721311475
Precision: 0.9514379592990391
Recall: 0.9508196721311475
F1 Score: 0.9508727683015862
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95        33
           1       0.93      0.96      0.95        28

    accuracy                           0.95        61
   macro avg       0.95      0.95      0.95        61
weighted avg       0.95      